In [1]:
import json
import pandas as pd

dataset_path = 'Tasks/NaturalLanguageInference/hellaswag/hellaswag_val.jsonl'


data = pd.read_json(path_or_buf=dataset_path, lines=True)

data.head()

,ind,activity_label,ctx_a,ctx_b,ctx,split,split_type,label,endings,source_id
0,24,Roof shingle removal,A man is sitting on a roof.,he,A man is sitting on a roof. he,val,indomain,3,"[is using wrap to wrap a pair of skis., is rip...",activitynet~v_-JhWjGDPHMY
1,92,Clean and jerk,A lady walks to a barbell. She bends down and ...,the lady,A lady walks to a barbell. She bends down and ...,val,zeroshot,3,"[swings and lands in her arms., pulls the barb...",activitynet~v_-lJS58hyo1c
2,106,Canoeing,Two women in a child are shown in a canoe whil...,the child and a different man,Two women in a child are shown in a canoe whil...,val,indomain,2,[are then shown paddling down a river in a boa...,activitynet~v_-xQvJmC2jhk
3,114,High jump,A boy is running down a track.,the boy,A boy is running down a track. the boy,val,zeroshot,2,"[runs into a car., gets in a mat., lifts his b...",activitynet~v_-zHX3Gdx6I4
4,116,High jump,The boy lifts his body above the height of a p...,the boy,The boy lifts his body above the height of a p...,val,zeroshot,1,"[turns his body around on the mat., gets up fr...",activitynet~v_-zHX3Gdx6I4


In [3]:
import torch
torch.cuda.empty_cache()
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.2.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Mar_21_19:15:46_PDT_2021
Cuda compilation tools, release 11.3, V11.3.58
Build cuda_11.3.r11.3/compiler.29745058_0
**********
CUDNN version: 8902
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 3080


In [2]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

from transformers import pipeline

model_id = "./Models/mistral/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, use_safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)


/home/jeremy/Documents/code/benchmark_LLM/benchmark_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.10s/it]


In [94]:


def format_prompt_choices(options):
    def rreplace(s, old, new, occurrence):
        li = s.rsplit(old, occurrence)
        return new.join(li)
    
    prompt_choices = None
    alfa = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    prompt_choices_list = [alfa[idx] for idx, x in enumerate(options)]
    prompt_choices = ', '.join(prompt_choices_list)

    prompt_choices = rreplace(prompt_choices,
                              ',',
                              ' or',
                              1)
    return prompt_choices

def decode_and_clean(generated_ids, tokenizer):
    '''
    - remove all whitespace
    - removes all fullstops
    - removes all commas
    - converts to int
    '''
    print(tokenizer.batch_decode(generated_ids)[0])
    return (tokenizer.batch_decode(generated_ids)[0].split('[/INST]')[-1].strip().replace('.','').replace(',','').replace('</s>',''))

def add_options(prompt_header, options):

    full_prompt = prompt_header

    def fill_option_template(option_num, option_sentence):
        alfa = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        template = f"""
            ### option {alfa[option_num]}:
            {option_sentence}

            """
        return template


    for idx, option in enumerate(options):
        full_prompt += fill_option_template(idx, option)


    
    return full_prompt



def mistal_pipeline(sample, model,tokenizer):
    target_sentence = sample['ctx']
    options = sample['endings']
    num_options = len(options)

    instruction = """
            Assume the role of an english teacher.
            You will be given a target sentence, followed by options. 
            You must choose a single option which best fits with the target sentence,
            if no option fits logically you must choose which option relates to the target sentence best when unsure.
            Ignore typos.
            You must only respond with your chosen option's corresponding letter and end the output.
            You must not explain or give reasoning for why the choice is right.
            Must return only the best choice event if unsure.
            """

    prompt_choices = format_prompt_choices(options)

    prompt_header = f"""
            {instruction}
            You must only output {prompt_choices}.
            
            
            ### Target sentence: 
            {target_sentence}...
            
            """




    full_prompt = add_options(prompt_header, options)


    sample = f"""
            {instruction}
            and you must only output A or B.
            
            
            ### Target sentence: 
            A man is typing on his keyboard. he...

            
            ### option A
            is shooting fireballs from his eyes.


            ### option B
            is doing work.
            """
    sample2 = f"""
            {instruction}
            and you must only output A, B or C.
            
            
            ### Target sentence: 
            A sitting down with food. he...

            
            ### option A
            is shooting fireballs from his eyes.


            ### option B
            is doing a bench press.

            
            ### option C
            is eating dinner.
            """
    messages = [
        {"role": "user", "content": sample},
        {"role": "assistant", "content": "B."},
        {"role": "user", "content": sample2},
        {"role": "assistant", "content": "C."},
        {"role": "user", "content": full_prompt}
    ]

    device = "cuda"
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)


    generated_ids = model.generate(model_inputs, temperature=0.3, max_new_tokens=2, do_sample=True)





    return decode_and_clean(generated_ids, tokenizer)

total_res = []

for idx, sample in data.iterrows():
    if idx < 10:
        res = mistal_pipeline(sample, model, tokenizer)
        total_res.append(res)


total_res

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] 
            
            Assume the role of an english teacher.
            You will be given a target sentence, followed by options. 
            You must choose a single option which best fits with the target sentence,
            if no option fits logically you must choose which option relates to the target sentence best when unsure.
            Ignore typos.
            You must only respond with your chosen option's corresponding letter and end the output.
            You must not explain or give reasoning for why the choice is right.
            Must return only the best choice event if unsure.
            
            and you must only output A or B.
            
            
            ### Target sentence: 
            A man is typing on his keyboard. he...

            
            ### option A
            is shooting fireballs from his eyes.


            ### option B
            is doing work.
             [/INST]B.</s> [INST] 
            
            Assume th

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] 
            
            Assume the role of an english teacher.
            You will be given a target sentence, followed by options. 
            You must choose a single option which best fits with the target sentence,
            if no option fits logically you must choose which option relates to the target sentence best when unsure.
            Ignore typos.
            You must only respond with your chosen option's corresponding letter and end the output.
            You must not explain or give reasoning for why the choice is right.
            Must return only the best choice event if unsure.
            
            and you must only output A or B.
            
            
            ### Target sentence: 
            A man is typing on his keyboard. he...

            
            ### option A
            is shooting fireballs from his eyes.


            ### option B
            is doing work.
             [/INST]B.</s> [INST] 
            
            Assume th

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] 
            
            Assume the role of an english teacher.
            You will be given a target sentence, followed by options. 
            You must choose a single option which best fits with the target sentence,
            if no option fits logically you must choose which option relates to the target sentence best when unsure.
            Ignore typos.
            You must only respond with your chosen option's corresponding letter and end the output.
            You must not explain or give reasoning for why the choice is right.
            Must return only the best choice event if unsure.
            
            and you must only output A or B.
            
            
            ### Target sentence: 
            A man is typing on his keyboard. he...

            
            ### option A
            is shooting fireballs from his eyes.


            ### option B
            is doing work.
             [/INST]B.</s> [INST] 
            
            Assume th

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] 
            
            Assume the role of an english teacher.
            You will be given a target sentence, followed by options. 
            You must choose a single option which best fits with the target sentence,
            if no option fits logically you must choose which option relates to the target sentence best when unsure.
            Ignore typos.
            You must only respond with your chosen option's corresponding letter and end the output.
            You must not explain or give reasoning for why the choice is right.
            Must return only the best choice event if unsure.
            
            and you must only output A or B.
            
            
            ### Target sentence: 
            A man is typing on his keyboard. he...

            
            ### option A
            is shooting fireballs from his eyes.


            ### option B
            is doing work.
             [/INST]B.</s> [INST] 
            
            Assume th

['D', 'D', 'C', 'A', 'B', 'B', 'B', 'A', 'A', 'B']

In [95]:
data.loc[:9,:]['label']

0    3
1    3
2    2
3    2
4    1
5    1
6    2
7    0
8    1
9    1
Name: label, dtype: int64

In [9]:
data['label'].unique()


array([3, 2, 1, 0])